# 参数管理
本节，我们将介绍以下内容：

访问参数，用于调试、诊断和可视化；

参数初始化；

在不同模型组件间共享参数。


我们首先看一下具有单隐藏层的多层感知机

In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2067],
        [0.3609]], grad_fn=<AddmmBackward0>)

参数访问

In [3]:
# net[2]指的是访问net所代表的模型中的第三个层
# .state_dict()是一个函数，它返回该层的状态字典（state dictionary），其中包含了层中所有可学习参数（如权重和偏差）的当前值。
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0261,  0.3525, -0.2899, -0.2765, -0.0821,  0.0569, -0.1954, -0.2978]])), ('bias', tensor([0.2537]))])


目标参数

注意，每个参数都表示为参数类的一个实例。 要对参数执行任何操作，首先我们需要访问底层的数值。 有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [4]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2537], requires_grad=True)
tensor([0.2537])


参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [5]:
net[2].weight.grad == None

True

一次性访问所有参数

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。 当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂， 因为我们需要递归整个树来提取每个子块的参数。 下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。

In [6]:
# *操作符有时被称为“解包”操作符。当与函数调用一起使用时，它会将列表、元组或其他可迭代对象中的元素解包（unpack）作为单独的参数传递给函数。
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示。

In [7]:
net.state_dict()['2.bias'].data

tensor([0.2537])

从嵌套块收集参数

将多个块相互嵌套，参数命名约定是如何工作的。 我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。

In [8]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
    # add_module 是 nn.Module 类的一个方法，用于向模型中添加子模块
    # 每个子模块都被分配了一个唯一的名称
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.1256],
        [0.1256]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的

In [9]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [10]:
rgnet[0][1][0].bias.data

tensor([-0.2656,  0.1920, -0.4882,  0.2681, -0.3721,  0.4722, -0.1861,  0.0240])

### 内置初始化

让我们首先调用内置的初始化器。 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。

In [11]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0081,  0.0099, -0.0274,  0.0096]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。

In [12]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)  # 初始化张量为常数值
        nn.init.zeros_(m.bias)
net.apply(init_constant)    # 递归地将函数应用于模型内的每个子模块
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对某些块应用不同的初始化方法。用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。
Xavier初始化方法是一种常用的权重初始化方法，旨在解决神经网络训练过程中梯度消失或爆炸的问题，通过根据输入和输出的连接数来合理地初始化权重，从而使得每层神经元的输出方差保持一致，避免信号在传播过程中因为权重过大或过小而消失或爆炸。

In [13]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4711,  0.6265, -0.3328, -0.3693])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化

In [14]:
def my_init(m):
    if type(m) == nn.Linear:
        # .named_parameters() 是 PyTorch 中用于获取神经网络模型中所有参数的方法。它会返回一个迭代器，该迭代器包含每个参数的名称及其对应的参数值。
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        # nn.init.uniform_ 是 PyTorch 中用于对张量进行均匀分布初始化的函数。它可以将输入张量初始化为指定范围内的均匀分布随机值。
        nn.init.uniform_(m.weight, -10, 10)
        # 大于等于 5 的元素保留，而小于 5 的元素置为 0
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000, -0.0000,  8.3256, -0.0000],
        [-0.0000, -6.3787,  6.4699,  0.0000]], grad_fn=<SliceBackward0>)

In [15]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  9.3256,  1.0000])

### 参数绑定

有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

参数绑定的目的可能包括：
1.  参数减少：如果网络中有重复的结构时，通过共享参数可以减少模型的大小。
2.  正则化：参数共享可以防止过拟合，特别是在小数据集上。
3.  特定任务的模式学习：在有些情况下，我们可能希望不同的功能部分学习到相同的模式。

当参数绑定时，梯度会发生什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

In [16]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
